In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


!pip install --upgrade gensim
!pip install sklearn-crfsuite
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
path = get_tmpfile("word2vec.model")
model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Requirement already up-to-date: gensim in /opt/conda/lib/python3.7/site-packages (3.8.3)
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 743 kB 857 kB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
/kaggle/input/chunking-preds/preds_pos.pkl
/kaggle/input/conll-2k-chunking/test.txt
/kaggle/input/conll-2k-chunking/train.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt
/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt


In [ ]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel
glovemod=loadGloveModel("../input/glove-global-vectors-for-word-representation/glove.6B.100d.txt")

In [ ]:
ps = PorterStemmer()
# ps.stem("unflattering")

In [ ]:
def featuremaker_withpos(sent, i):
#     print(len(sent),end=' ')
#     print(i)
    word = sent[i][0]
    wembed=glovemod.get(word.lower(),glovemod['.'])
    features = {
        'bias': 1.0,
        #'word': glovemod.get(word.lower(),glovemod['.']),
        'length':len(word),
        'suffix': word.replace(ps.stem(word),""),
        'allup': word.isupper(),
        'cpos':sent[i][1],
        #'word.istitle()': word.istitle(),
        'num': word.isdigit(),
        #'postag': postag,
    }
    for j in range(wembed.shape[0]):
        features["cw"+str(j)]=wembed[j]
    
    if i > 0:
        features['BEG']=False
        features['ppos']=sent[i-1][1]
        features['cp']=sent[i-1][1]+"|"+sent[i][1]
#         print(i)
#         print(len(sent))
        pword = sent[i-1][0]
        #postag1 = sent[i-1][1]
#         features.update({
#             'bias': 1.0,
#         #'word': glovemod.get(word.lower(),glovemod['.']),
        
#         'pword': pword.replace(ps.stem(pword),""),
#         'allup': pword.isupper(),
#         #'word.istitle()': word.istitle(),
#         'num': pword.isdigit(),
#         #'postag': postag,
#         })
        pwembed=glovemod.get(pword.lower(),glovemod['.'])
        for j in range(pwembed.shape[0]):
            features["pw"+str(j)]=pwembed[j]
    else:
        features['cp']="$"+"|"+sent[i][1]
        features['BEG'] = True
        features['FIRSTW'] = False
     
    if i > 1:
        features['FIRST']=False
        features['pppos']=sent[i-2][1]
        ppword = sent[i-2][0]
        #postag1 = sent[i-1][1]
#         features.update({
#             'bias': 1.0,
#         #'word': glovemod.get(word.lower(),glovemod['.']),
        
#         'pword': pword.replace(ps.stem(pword),""),
#         'allup': pword.isupper(),
#         #'word.istitle()': word.istitle(),
#         'num': pword.isdigit(),
#         #'postag': postag,
#         })
        ppwembed=glovemod.get(ppword.lower(),glovemod['.'])
        for j in range(ppwembed.shape[0]):
            features["ppw"+str(j)]=ppwembed[j]
    elif i==1:
        features['BEG'] = False
        features['FIRSTW'] =True
    
    if i < len(sent)-1:
        features['on']=sent[i][1]+"|"+sent[i+1][1]
        features['EOS']=False
        features['npos']=sent[i+1][1]
        nword = sent[i+1][0]
        #postag1 = sent[i+1][1]
        nwembed=glovemod.get(nword.lower(),glovemod['.'])
        for j in range(nwembed.shape[0]):
            features["nw"+str(j)]=nwembed[j]
    else:
        features['on']=sent[i][1]+"|"+"$"
        features['LBO']=False
        features['EOS'] = True
    if i < len(sent)-2:
        features['LBO']=False
        features['nnpos']=sent[i+2][1]
        nnword = sent[i+2][0]
    #postag1 = sent[i+2][1]
        nnwembed=glovemod.get(nnword.lower(),glovemod['.'])
        for j in range(nnwembed.shape[0]):
            features["nnw"+str(j)]=nnwembed[j]
    elif i==len(sent)-2:
        features['LBO']=True #last but one
        features['EOS'] = False
    
    return features

In [ ]:
allsents=[]
currsent=[]
alllabels=[]
currlabels=[]
with open('../input/conll-2k-chunking/train.txt') as f:
    for line1 in f:
        line=line1.strip()
        if line=='':
            allsents.append(currsent)
            currsent=[]
            alllabels.append(currlabels)
            currlabels=[]
        else:
            word,pos,label=line.split(' ')
            label=label.split('-')[0]
            currsent.append((word,pos))
            currlabels.append(label)

In [ ]:
featurevecs=[[featuremaker_withpos(x, i) for i in range(len(x))] for x in allsents]


In [ ]:
#print(set([y[1] for x in allsents for y in x]))

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(featurevecs, alllabels)

In [ ]:
del featurevecs

In [3]:

allsentstest=[]
currsenttest=[]
alllabelstest=[]
currlabelstest=[]
with open('../input/conll-2k-chunking/test.txt') as f:
    for line1 in f:
        line=line1.strip()
        if line=='':
            allsentstest.append(currsenttest)
            currsenttest=[]
            alllabelstest.append(currlabelstest)
            currlabelstest=[]
        else:
            word,pos,label=line.split(' ')
            label=label.split('-')[0]
            currsenttest.append((word,pos))
            currlabelstest.append(label)

In [ ]:
featurevecstest=[[featuremaker_withpos(x, i) for i in range(len(x))] for x in allsentstest]

In [ ]:
labels = list(crf.classes_)
# labels.remove('O')
allpreds = crf.predict(featurevecstest)
sklearn_crfsuite.metrics.flat_f1_score(alllabelstest, allpreds,
                      average='weighted', labels=labels)

In [ ]:
print(sklearn_crfsuite.metrics.flat_accuracy_score(alllabelstest, allpreds))

In [ ]:
# print(sklearn_crfsuite.flat_classification_report(alllabelstest, allpreds))

In [ ]:
import joblib
joblib.dump(allpreds, 'preds_pos.pkl') 
joblib.dump(alllabelstest,'preds.pkl')


In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def get_report(y_true, y_pred, classes):
    clf_report = classification_report(y_true, y_pred, labels=classes, zero_division=0)
    clf_report = clf_report.replace('\n\n', '\n')
    clf_report = clf_report.replace('micro avg', 'micro_avg')
    clf_report = clf_report.replace('macro avg', 'macro_avg')
    clf_report = clf_report.replace('weighted avg', 'weighted_avg')
    clf_report = clf_report.replace(' / ', '/')
    lines = clf_report.split('\n')

    class_names, plotMat, support = [], [], []
    for line in lines[1:]:
        t = line.strip().split()
        if len(t) < 2:
            continue
        v = [float(x) for x in t[1: len(t) - 1]]
        if len(v) == 1 : v = v * 3
        support.append(int(t[-1]))
        class_names.append(t[0])
        plotMat.append(v)
    plotMat = np.array(plotMat)
    support = np.array(support)
    return class_names, plotMat, support

def get_scores(y_true, y_pred, classes):
    correct, wrong = {}, {}
    for tag in classes:
        correct[tag] = 0
        wrong[tag] = 0
        
    for tag, pred in zip(y_true, y_pred):
        if tag == pred:
            correct[tag] += 1
        else:
            wrong[tag] += 1
            
    scores = []
    total = len(y_true)
    for tag in classes:
        cur = np.array([correct[tag], wrong[tag]])
        scores.append(cur / total)
    return np.array(scores)
    
def plot_confusion_matrix(classes, mat, normalize=True, cmap=plt.cm.Blues):
    cm = np.copy(mat)
    title = 'Confusion Matrix (without normalization)'
    if normalize:
        cm = cm.astype('float') / np.sum(cm, axis=1, keepdims=True)
        title = title.replace('without', 'with')
    plt.clf()    
    fig, ax = plt.subplots(figsize=(20,10))
    ax.set_title(title, y=-0.06, fontsize=22)
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.clim(vmin=0.0, vmax=1.0)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = np.max(cm) / 2
    thresh = 1 / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            color = "white" if (cm[i, j] > thresh) else "black"
            plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color=color)
    plt.ylabel('True label',fontsize=22)
    plt.xlabel('Predicted label', fontsize=22)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', bbox_inches="tight", transparent=True)
    plt.show()
    
def plot_clf_report(classes, plotMat, support, cmap=plt.cm.Blues):
    title = 'Classification Report'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    yticklabels = ['{0} ({1})'.format(classes[idx], sup) for idx, sup in enumerate(support)]
    plt.clf()
    fig, ax = plt.subplots(figsize=(20,10))
    ax.set_title(title, y=-0.06, fontsize=22)
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    ax.xaxis.set_tick_params(labelsize=18)
    ax.yaxis.set_tick_params(labelsize=14)
    plt.imshow(plotMat, interpolation='nearest', cmap=cmap, aspect='auto')
    plt.clim(vmin=0.0, vmax=1.0)
    plt.colorbar()
    plt.xticks(np.arange(3), xticklabels, rotation=0)
    plt.yticks(np.arange(len(classes)), yticklabels)
    thresh = np.max(plotMat) / 2
    thresh = 1 / 2
    for i in range(plotMat.shape[0]):
        for j in range(plotMat.shape[1]):
            color = "white" if (plotMat[i, j] > thresh) else "black"
            plt.text(j, i, format(plotMat[i, j], '.2f'), horizontalalignment="center", color=color, fontsize=14)

    plt.xlabel('Metrics',fontsize=22)
    plt.ylabel('Classes',fontsize=22)
    plt.tight_layout()
    plt.savefig('classification_report.png', bbox_inches="tight", transparent=True)
    plt.show()
    
def plot_tag_scores(classes, scores, normalize=True):
    plt.clf()
    width = 0.45
    fig, ax = plt.subplots(figsize=(20,10))
    ax.xaxis.set_tick_params(labelsize=18, rotation=25)
    ax.yaxis.set_tick_params(labelsize=18)
    range_bar1 = np.arange(len(classes))
    rects1 = ax.bar(range_bar1, tuple(scores[:, 0]), width, color='b')
    rects2 = ax.bar(range_bar1 + width, tuple(scores[:, 1]), width, color='r')

    ax.set_ylabel('Scores',fontsize=22)
    ax.set_title('Tag scores', fontsize=22)
    ax.set_xticks(range_bar1 + width / 2)
    ax.set_xticklabels(classes)

    ax.legend((rects1[0], rects2[0]), ('Correct', 'Wrong'), fontsize=20)
    plt.legend()
    plt.savefig('tag_scores.png', bbox_inches="tight", transparent=True)
    plt.show()

In [ ]:
y_true=[x for y in alllabelstest for x in y]
y_pred=[x for y in  allpreds for x in y]
classes = ['B','I','O']
class_names, report, support = get_report(y_true, y_pred, classes)
cm = confusion_matrix(y_true, y_pred, labels=classes)
scores = get_scores(y_true, y_pred, classes)
plot_clf_report(class_names, report, support)
plot_confusion_matrix(classes, cm)
plot_tag_scores(classes, scores)


In [11]:
import pickle
errswords=[]
correctlabels=[]
errlabels=[]
# ff=open('../input/chunking-preds/preds_pos.pkl','rb')
# allpreds=pickle.load(ff)
for x,y,z in zip(allsentstest,alllabelstest,allpreds):
    for i in range(len(y)):
        if y[i]!=z[i]:
#             print('lol')
            errswords.append([x[i-1] if i>0 else '##',x[i],x[i+1] if i<len(x)-1 else '##'])
            correctlabels.append([y[i-1] if i>0 else '##',y[i],y[i+1] if i<len(y)-1 else '##'])
            errlabels.append([z[i-1] if i>0 else '##',z[i],z[i+1] if i<len(z)-1 else '##'])

In [12]:
print(len(errswords))
[print((str(i[0][0])  if i[0][0]!=None else '##' )+' '+str(i[1][0])+' '+(str(i[2][0])  if i[2][0]!=None else '##' )+' ( correct: '+j[0]+' '+j[1]+' '+j[2]+ ' predicted: '+k[0]+' '+k[1]+' '+k[2]+' )') for i,j,k in zip(errswords[:100],correctlabels[:100],errlabels[:100])]

1878
's two major ( correct: B I I predicted: B B I )
leading edges for ( correct: I I B predicted: I B B )
aerospace , electronics ( correct: I I I predicted: I O B )
, electronics , ( correct: I I I predicted: O B O )
electronics , automotive ( correct: I I I predicted: B O B )
, automotive and ( correct: I I I predicted: O B I )
graphics concern . ( correct: I I O predicted: I B O )
is newly created ( correct: B I I predicted: B B I )
been executive vice ( correct: I B I predicted: I I I )
development and property ( correct: B O B predicted: B I I )
and property management ( correct: O B I predicted: I I I )
about 60%-held by ( correct: B I B predicted: B B B )
bank had earnings ( correct: I B B predicted: I I I )
had earnings of ( correct: B B B predicted: I I B )
loan and real ( correct: I I I predicted: I O B )
and real estate ( correct: I I I predicted: O B I )
had operating profit ( correct: B B I predicted: B I B )
operating profit of ( correct: B I B predicted: I B B )
bank t

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]